In [ ]:
import re
from os import path
from pprint import pprint

REF = path.expanduser('~/github/pthu/tfbuilder/tlg_to_tf')

f1 = open(f'{REF}/data/refs_1.txt', encoding='utf-8')
f2 = open(f'{REF}/data/refs_2.txt', encoding='utf-8')

def splitLine(line):
    first_space = line.find(' ')
    return [line[:first_space].strip(), 
            line[first_space:].strip()]

def authorMetadata(file):
    author_dict = {}
    key = None
    authorFound = False
    for l in file.readlines():
        line = l.strip()
        if line.startswith('key') and len(line.split()) == 3:
            authorFound = False
        elif authorFound:
            code, content = splitLine(line)
            author_dict[key].update({code: content})
        elif line.startswith('key') and len(line.split()) == 2:
            authorFound = True
            _, key = line.split()
            author_dict[key] = {}
    return author_dict


def workMetadata(file):
    workRE    = re.compile(r'^([0-9]+){4} (\w+){3}$')
    work_dict = {}
    start     = False
    workName  = ''
    nameFound = False
    workFound = False
    
    for l in file.readlines():
        line = l.strip()
        if len(line) == 4 and line.isdigit():
            workFound = False
        elif workRE.fullmatch(line):
            if start:
                work_dict[work] = (name, workName)
            start = True
            work = tuple(line.split(' '))
            workFound = True
            nameFound = False
            workName  = ''
            continue
        elif workFound:
            if not nameFound:
                if ',' in line:
                    name = line[:line.find(',')]
                else:
                    name = line
                nameFound = True
                workName += line
            else:
                workName += ' ' + line        
        else:
            continue
    return work_dict


def bibliography(file): ##TODO!
    bibl_dict = {}
    author = None
    workFound = False
    for line in file.readlines():
        line = line.strip()
        if line.replace(' ', '').isdigit() and \
                len(line.replace(' ', '')) == 7:
            workFound = True
            author = '-'.join(line.split())
            ref = ''
        elif line.replace(' ', '').isdigit() and \
                  len(line.replace(' ', '')) == 4:
            workFound = False
            if author:
                bibl_dict[author] = ref[:-1] + '.'
        elif workFound:
            ref += line.strip() + ' '
        
        else:
            continue
    return bibl_dict
            
        

author_result = authorMetadata(f2)
works_result = workMetadata(f1)
bibl_result = bibliography(f1)
# pprint(author_result)
pprint(works_result)
# pprint(bibl_result)

# print(len(works_result))
# print(len(author_result))
# pprint(author_result)

f1.close()
f2.close()

In [ ]:
l = '0002 004 '
print(l[:4])
ln = l.replace(' ', '')
print(ln)
if ln.isdigit() and len(ln) == 7:
    print('yes')